In [2]:
!pip install numpy

Looking in indexes: https://artifacts.jpmchase.net/artifactory/api/pypi/pypi/simple
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     ----- ---------------------------------- 1.8/12.8 MB 7.2 MB/s eta 0:00:02
     ------------------------------- ------- 10.5/12.8 MB 23.0 MB/s eta 0:00:01
     ----------------------------------- --- 11.5/12.8 MB 22.0 MB/s eta 0:00:01
     ----------------------------------- --- 11.5/12.8 MB 22.0 MB/s eta 0:00:01
     ----------------------------------- --- 11.8/12.8 MB 12.6 MB/s eta 0:00:01
     ------------------------------------ -- 12.1/12.8 MB 10.4 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 9.0 MB/s eta 0:00:00


In [1]:
import numpy as np

In [2]:
def sigmoid(z):
  return 1.0 / (1 + np.exp(-z))

In [3]:
def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))

In [4]:
def sigmoid_derivative(x):
    return x * (1 - x)

In [6]:
def initialized_network(input_size: int, hidden_layers: list, output_size: int):
    layers = [input_size] + hidden_layers + [output_size]
    weights = []
    biases = []
    print(f' initialize_network {layers=}')
    
    l = len(layers) - 1

    for i in range(l):
        
        weight = np.random.uniform(-1, 1, (layers[i], layers[i+1]))
        bias = np.random.uniform(-1, 1, (layers[i+1],))
        print(f' #{i+1}/{l} {weight=}')
        print(f'   {bias=}\n')
        weights.append(weight)
        biases.append(bias)

    return weights, biases
        

In [7]:
def train_network(td, w, b, lr=0.1, e=1000):
    for epoch in range(e):
        total_error = 0
        for input_val, expected_output in td:
            total_error += train_once(input_val, expected_output, lr, w, b)
        if epoch % 1000 ==0:
            print(f"Epoch {epoch}, Total Error: {total_error}")

    return w, b
            

In [8]:
def forward_pass(input_data, weights, biases):
    activations = [input_data]
    for i in range(len(weights)):
        # on error: likely due to the input shape not matching the input nodes.
        input_data = np.dot(input_data, weights[i]) + biases[i]
        input_data = sigmoid(input_data)
        activations.append(input_data)
    return activations

In [39]:

def forward_pass_one(input_data, weights, biases):
    # # Generalized forward pass through all layers
    # for i in range(len(weights)):
    #     input_data = np.dot(input_data, weights[i]) + biases[i]
    #     input_data = sigmoid(input_data)
    # return input_data
    return forward_pass(input_data, weights, biases)[-1]


In [9]:
def train_once(input_val, expected_output, learning_rate, weights, biases):
    """
    # Example of adjusting training data to include negative example flag
    training_data = [
        # Positive examples: ((input_features), (target_output, False))
        ((input_features_for_example_1), (target_output_for_example_1, False)),
        # Negative examples: ((input_features), (target_output, True))
        ((input_features_for_negative_example_1), (target_output_for_negative_example_1, True)),
    ]

    """
    # Forward pass: Store activations for each layer
    activations = forward_pass(input_val, weights, biases)

    # Backward pass
    # Calculate output error
    output_error = expected_output - activations[-1]
    total_error = np.sum(output_error ** 2)

    # Calculate gradient for output layer
    d_error = output_error * sigmoid_derivative(activations[-1])

    for i in reversed(range(len(weights))):
        # Calculate error for the current layer
        d_weights = np.outer(activations[i], d_error)
        d_biases = d_error

        # Update weights and biases
        weights[i] += learning_rate * d_weights
        # cannot be: `biases[i] += learning_rate * d_biases` due to numpy
        biases[i] = biases[i] + (learning_rate * d_biases)

        # Propagate the error backward
        if i > 0:
            d_error = np.dot(d_error, weights[i].T) * sigmoid_derivative(activations[i])
    return total_error

In [10]:
def dictify(word):
    thedict = {}
    for i, letter in enumerate(word):
        thedict[letter] = (i+1)/len(word)
    return thedict

In [11]:
bits = dictify("catdog")
bits

{'c': 0.16666666666666666,
 'a': 0.3333333333333333,
 't': 0.5,
 'd': 0.6666666666666666,
 'o': 0.8333333333333334,
 'g': 1.0}

In [12]:
tds = [
    (
        (0.16666666666666666,0.3333333333333333, 0.5),
        (0.6666666666666666,0.8333333333333334, 1.0),
    ),
    (
        (0.6666666666666666,0.8333333333333334, 1.0),
        (0.16666666666666666,0.3333333333333333, 0.5),
    )
]

In [109]:
ws, bs = initialized_network(3,[3],3)
train_network(tds, ws, bs)

 initialize_network layers=[3, 3, 3]
 #1/2 weight=array([[-0.08467469,  0.66524532, -0.12376302],
       [-0.02059697, -0.81449916,  0.20397161],
       [-0.44586953,  0.13137513, -0.17495557]])
   bias=array([0.69934757, 0.38707912, 0.96299195])

 #2/2 weight=array([[-0.26217839, -0.19768127, -0.94532358],
       [-0.02115293, -0.68977889, -0.75805636],
       [-0.1162863 ,  0.78762624,  0.26588285]])
   bias=array([ 0.9303869 , -0.44821179,  0.14953115])

Epoch 0, Total Error: 0.9005848636463121


([array([[ 0.37858231,  1.36688834, -1.06724162],
         [ 0.42051806, -0.19149396, -0.69232114],
         [-0.02689648,  0.6757425 , -1.02406248]]),
  array([[-0.57112502,  0.07903734, -0.43558203],
         [-0.81688282, -0.85464771, -0.63478037],
         [ 0.34204181,  1.78740261,  1.49348678]])],
 [array([ 0.56649571, -0.08474784,  1.246107  ]),
  array([0.49913064, 0.07700377, 1.08773553])])

In [82]:
cat_numbers = (bits["c"], bits["a"], bits["t"])
fp = forward_pass(cat_numbers, ws, bs)
stib = {v:k for k,v in bits.items()}
print(f"stib: {stib}")
print(f"fp: {fp}")

stib: {0.16666666666666666: 'c', 0.3333333333333333: 'a', 0.5: 't', 0.6666666666666666: 'd', 0.8333333333333334: 'o', 1.0: 'g'}
fp: [(0.16666666666666666, 0.3333333333333333, 0.5), array([0.74902055, 0.46451413, 0.8523532 ]), array([0.45266945, 0.57911559, 0.75855356])]


In [33]:
def closest_match(value, values_in, values_out):
    fp = forward_pass(value, ws, bs)
    for i in range(len(ws)):
        value = np.dot(value, ws[i]) + bs[i]
        value = sigmoid(value)
    final_output = fp[0]
    closest_numb = min(values_in.values(), key=lambda x: abs(x - final_output))
    predicted_letter = values_out[closest_numb]
    return predicted_letter

In [78]:

def predict(input_val, weights, biases, letter_to_num, num_to_letter):
    # Ensure input is in the correct format (numpy array)
    input_val = np.array(input_val)

    v = forward_pass_one(input_val, weights, biases)
    # Generalized forward pass through all layers
    for i in range(len(weights)):
        input_val = np.dot(input_val, weights[i]) + biases[i]
        input_val = sigmoid(input_val)
    # assert v == input_val
    # Convert the final output to the closest letter
    # Here, we're assuming the output is a single number. Adjust as needed for multiple outputs.
    # final_output = v[0]  # Assuming single output for simplicity
    res = ''
    for final_output in v:
        closest_num = min(letter_to_num.values(), key=lambda x: abs(x - final_output))
        predicted_letter = num_to_letter[closest_num]
        # print(predicted_letter)
        res += predicted_letter
    return res


In [118]:
predict(cat_numbers, ws, bs, bits, stib)

'dog'

In [111]:
ws, bs

([array([[ 0.37858231,  1.36688834, -1.06724162],
         [ 0.42051806, -0.19149396, -0.69232114],
         [-0.02689648,  0.6757425 , -1.02406248]]),
  array([[-0.57112502,  0.07903734, -0.43558203],
         [-0.81688282, -0.85464771, -0.63478037],
         [ 0.34204181,  1.78740261,  1.49348678]])],
 [array([ 0.56649571, -0.08474784,  1.246107  ]),
  array([0.49913064, 0.07700377, 1.08773553])])

In [87]:
closest_match(.1, bits, stib)

'c'

In [88]:
fp

[(0.16666666666666666, 0.3333333333333333, 0.5),
 array([0.74902055, 0.46451413, 0.8523532 ]),
 array([0.45266945, 0.57911559, 0.75855356])]

In [89]:
for letter in fp[0]:
    print(closest_match(letter, bits, stib))

c
a
t


In [90]:
for letter in fp[2]:
    print(closest_match(letter, bits, stib))

t
t
o


In [117]:
train_network(tds, ws, bs)

Epoch 0, Total Error: 0.007996256817873554


([array([[ 0.73673958,  2.10778114, -2.25578938],
         [ 0.72377971,  0.35239236, -1.61815317],
         [ 0.22146956,  1.02262233, -1.68717877]]),
  array([[-1.03499628, -0.1384617 , -0.68205288],
         [-1.95576111, -1.46639346, -1.51979331],
         [ 2.26849983,  2.9332342 ,  3.31789246]])],
 [array([ 0.23712203, -1.26678675,  2.82240144]),
  array([0.52627935, 0.1604698 , 1.34076506])])

In [104]:
fp

[(0.16666666666666666, 0.3333333333333333, 0.5),
 array([0.74902055, 0.46451413, 0.8523532 ]),
 array([0.45266945, 0.57911559, 0.75855356])]